In [1]:
import os
import json

import pandas as pd

from sklearn.feature_selection import RFE

import xgboost as xgb

In [2]:
os.chdir("..")
os.chdir("..")

# Parameters

In [3]:
antibiotic_dictionary_file = open("data/antibiotics.json", "r")
antibiotic_dictionary = json.load(antibiotic_dictionary_file)
antibiotics = antibiotic_dictionary["Escherichia coli"]

In [4]:
file = "data/processed/raw/train_e_coli_driams.csv"

train_bac = pd.read_csv(file)
train_bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9994,9995,9996,9997,9998,9999,Ciprofloxacin,Ceftriaxone,Piperacillin-Tazobactam,Cefepime
0,0.001530,0.001362,0.001367,0.001134,0.001164,0.000875,0.001573,0.000936,0.001014,0.001009,...,0.012174,0.013379,0.013813,0.019293,0.019271,0.010970,0.0,0.0,0.0,0.0
1,0.012835,0.015030,0.016545,0.019867,0.021450,0.018100,0.016463,0.012731,0.012761,0.013359,...,0.022339,0.024486,0.018417,0.022405,0.024961,0.029598,1.0,1.0,0.0,1.0
2,0.036591,0.038338,0.040873,0.047490,0.052187,0.055778,0.055738,0.047467,0.046334,0.045266,...,0.080196,0.091884,0.095694,0.081280,0.082773,0.095949,0.0,1.0,0.0,1.0
3,0.017140,0.017536,0.018672,0.018508,0.017872,0.019658,0.020053,0.020572,0.018712,0.018686,...,0.025351,0.026505,0.024017,0.024147,0.030135,0.025802,0.0,0.0,0.0,0.0
4,0.003033,0.003481,0.002779,0.004176,0.004205,0.002828,0.004182,0.003030,0.002802,0.003335,...,0.041165,0.043797,0.061598,0.038835,0.049534,0.073161,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,0.017158,0.018512,0.018706,0.020291,0.020911,0.020753,0.019474,0.018620,0.018705,0.018179,...,0.053464,0.057933,0.055251,0.051033,0.054449,0.082976,0.0,0.0,0.0,0.0
3687,0.024098,0.023388,0.023772,0.025077,0.024409,0.024004,0.026094,0.027139,0.027487,0.023848,...,0.019453,0.022214,0.023768,0.017426,0.018883,0.009809,1.0,1.0,0.0,1.0
3688,0.010605,0.013530,0.013734,0.014226,0.014336,0.014168,0.013768,0.012725,0.010554,0.012690,...,0.003765,0.008078,0.006595,0.004730,0.002975,0.003650,0.0,0.0,1.0,0.0
3689,0.007466,0.008374,0.008350,0.008565,0.008374,0.007656,0.008862,0.009450,0.010870,0.010106,...,0.017319,0.016282,0.018666,0.019666,0.018883,0.029474,0.0,0.0,0.0,0.0


# Attribute Selection

In [5]:
x_train = train_bac[train_bac.columns.drop(list(train_bac.filter(regex='[^0-9]')))]
y_train = train_bac[antibiotics]

In [6]:
profile = train_bac[antibiotics]
profile

,Ciprofloxacin,Ceftriaxone,Piperacillin-Tazobactam,Cefepime
0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,1.0
2,0.0,1.0,0.0,1.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
3686,0.0,0.0,0.0,0.0
3687,1.0,1.0,0.0,1.0
3688,0.0,0.0,1.0,0.0
3689,0.0,0.0,0.0,0.0


In [7]:
selector = RFE(
    estimator=xgb.XGBClassifier(objective="binary:logistic"),
    n_features_to_select=100,
    step=5,
)
selector.fit(x_train, y_train)